In [16]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import PIL.Image as Image
import numpy as np
import os
import random


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])


train_data = datasets.ImageFolder(root=r"C:\Users\ujjwal\.cache\kagglehub\datasets\jhislainematchouath\soil-types-dataset\versions\1\Dataset\Train", transform=transform)
test_data = datasets.ImageFolder(root=r"C:\Users\ujjwal\.cache\kagglehub\datasets\jhislainematchouath\soil-types-dataset\versions\1\Dataset\test", transform=transform)


batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


class_names = train_data.classes
print("Classes:", class_names)


Classes: ['Alluvial soil', 'Black Soil', 'Clay soil', 'Red soil']


In [ ]:
class SoilCNN(nn.Module):
    def __init__(self, num_classes=4):  
        super(SoilCNN, self).__init__()

       
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        
        self.fc1 = nn.Linear(128 * 16 * 16, 128)  
        self.fc2 = nn.Linear(128, num_classes)

        
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  
        x = self.pool(self.relu(self.conv2(x)))  
        x = self.pool(self.relu(self.conv3(x)))  

        
        x = x.view(x.size(0), -1)

        x = self.relu(self.fc1(x))  
        x = self.dropout(x)  
        x = self.fc2(x)  

        return x


device = torch.device("cpu")  
model = SoilCNN(num_classes=4).to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training Finished!")

Epoch [1/10], Loss: 0.7236
Epoch [2/10], Loss: 0.4429
Epoch [3/10], Loss: 0.3426
Epoch [4/10], Loss: 0.2919
Epoch [5/10], Loss: 0.2164
Epoch [6/10], Loss: 0.1823
Epoch [7/10], Loss: 0.1474
Epoch [8/10], Loss: 0.1303
Epoch [9/10], Loss: 0.1481
Epoch [10/10], Loss: 0.1224
Training Finished!


In [10]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 94.41%


In [ ]:
class_labels = ["alluvial", "black", "clay", "red"]


def get_random_image(folder):
   
    if not os.path.exists(folder):
        print(f"Error: Folder '{folder}' does not exist.")
        return None
    
    
    subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]
    if not subfolders:
        print("No subfolders found in dataset!")
        return None
    
    
    random_class = random.choice(subfolders)
    class_path = os.path.join(folder, random_class)
    
    
    images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if not images:
        print(f"No images found in '{class_path}'")
        return None
    
   
    random_image = random.choice(images)
    return os.path.join(class_path, random_image)


def predict_image(image_path, model):
    """ Predicts the soil type of a given image and prints its filename. """
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    
    image_name = os.path.basename(image_path)

    
    model.eval()
    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item() 
        predicted_label = class_labels[predicted_class] 

    print(f"{image_name} -> Predicted Soil Type: {predicted_label.capitalize()} Soil")


test_folder = r"C:\Users\ujjwal\.cache\kagglehub\datasets\jhislainematchouath\soil-types-dataset\versions\1\Dataset\test"


image_path = get_random_image(test_folder)

if image_path:
    predict_image(image_path, model)

image16 (2).jpeg -> Predicted Soil Type: Clay Soil


In [ ]:
torch.save(model.state_dict(), "soil_classification_model.pth")


loaded_model = SoilCNN(num_classes=4)
loaded_model.load_state_dict(torch.load("soil_classification_model.pth", map_location=torch.device('cpu')))
loaded_model.to(device)

SoilCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
)